In [2]:
import numpy as np
import mlflow
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.neural_network import MLPClassifier


class NeuralNetwork:
    def __init__(self, hidden_layer_sizes=(100,), max_iter=200, random_state=42):
        self.hidden_layer_sizes = hidden_layer_sizes
        self.max_iter = max_iter
        self.random_state = random_state
        self.model = None
        self.tracking = mlflow.set_tracking_uri("http://localhost:5000")
        self.experiment = mlflow.set_experiment("NeuralNetwork")

    def _build_model(self, input_shape, output_shape):
        model = MLPClassifier(hidden_layer_sizes=self.hidden_layer_sizes, max_iter=self.max_iter, 
                              random_state=self.random_state)
        return model

    def fit(self, X_train, y_train, X_val, y_val):
        with mlflow.start_run():

            mlflow.log_param("hidden_layer_sizes", self.hidden_layer_sizes)
            mlflow.log_param("max_iter", self.max_iter)
            mlflow.log_param("random_state", self.random_state)

            input_shape = X_train.shape[1:]
            output_shape = 1
            self.model = self._build_model(input_shape, output_shape)

            self.model.fit(X_train, y_train)

            y_train_pred = self.model.predict(X_train)
            train_accuracy = accuracy_score(y_train, y_train_pred)
            train_precision = precision_score(y_train, y_train_pred)
            train_recall = recall_score(y_train, y_train_pred)
            train_f1 = f1_score(y_train, y_train_pred)

            y_val_pred = self.model.predict(X_val)
            val_accuracy = accuracy_score(y_val, y_val_pred)
            val_precision = precision_score(y_val, y_val_pred)
            val_recall = recall_score(y_val, y_val_pred)
            val_f1 = f1_score(y_val, y_val_pred)

            mlflow.log_metric("train_accuracy", train_accuracy)
            mlflow.log_metric("train_precision", train_precision)
            mlflow.log_metric("train_recall", train_recall)
            mlflow.log_metric("train_f1", train_f1)
            mlflow.log_metric("val_accuracy", val_accuracy)
            mlflow.log_metric("val_precision", val_precision)
            mlflow.log_metric("val_recall", val_recall)
            mlflow.log_metric("val_f1", val_f1)

            mlflow.sklearn.log_model(self.model, "model")

    def predict(self, X_test):
        y_pred = self.model.predict(X_test)
        return y_pred.astype(int)


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

df2 = pd.read_csv('dados.csv')
X = df2.drop('downgrade', axis=1)
y = df2['downgrade']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
nn = NeuralNetwork()

# treinando o modelo
nn.fit(X_train, y_train, X_test, y_test)

# fazendo as previsões no conjunto de teste
y_pred = nn.predict(X_test)
y_pred

/home/jsalnx01/.pyenv/versions/3.10.6/envs/mlops/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


array([1, 1, 1, ..., 1, 1, 1])

In [5]:
prop_down = np.mean(y_pred)
prop_down.round(2)

0.86